In [1]:
import pandas as pd
import sqlite3

In [2]:
# Conexão database SQLite
consqlite = sqlite3.connect('C:/Users/matheus.candido/sqlite3/exploracao_petroleo_brasil.db', timeout=3)


In [3]:
# ETL Query d_situacao

query_d_situacao = f"""
SELECT DISTINCT 
       CASE WHEN SITUACAO LIKE 'ABANDONADO TEMPORARIAMENTE SEM MONITORAMENTO' THEN 'AB44T' WHEN SITUACAO LIKE 'EM OBSERVAÇÃO' THEN 'EM13A' 
   		 	WHEN SITUACAO LIKE 'EM COMPLETAÇÃO' THEN 'EM14A' WHEN SITUACAO LIKE 'EQUIPADO AGUARDANDO INÍCIO DE OPERAÇÃO' THEN 'EQ38A' 
   		 	ELSE SUBSTR(SITUACAO, 0, 3) || LENGTH (SITUACAO) || SUBSTR(SITUACAO, -1, 1) END AS COD_SITUACAO,
	   SITUACAO 
FROM T_DATAHUB
WHERE COD_SITUACAO IS NOT NULL
ORDER BY COD_SITUACAO
"""

d_situacao_df = pd.read_sql_query(query_d_situacao, consqlite)

In [4]:
d_situacao_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   COD_SITUACAO  25 non-null     object
 1   SITUACAO      25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes


In [5]:
# Create table (DDL) SQLite d_situacao

consqlite.execute('DROP TABLE D_SITUACAO')
consqlite.execute(f"""
CREATE TABLE IF NOT EXISTS D_SITUACAO (
	COD_SITUACAO VARCHAR(20) PRIMARY KEY,
	SITUACAO VARCHAR(100) NOT NULL
)
""")

In [6]:
# Insert de dados d_situacao de dataframe para database

d_situacao_df.to_sql('D_SITUACAO', consqlite, if_exists='append', index=False)

In [7]:
# Informações de database d_situacao

d_situacao_db = pd.read_sql_query('SELECT * FROM D_SITUACAO', consqlite)
d_situacao_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   COD_SITUACAO  25 non-null     object
 1   SITUACAO      25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes
